# **Modélisation statistique**
Nous allons mener des régressions linéaires de la production électrique éolienne (données API RTE) sur les données météorologiques de vents (données API ...)

Dans un premier temps, nous allons créer un data frame qui contiendra la moyenne nationale de la production électrique éolienne ainsi que la moyenne des vents nationaux, heure par heure pour l'année 2021.
Ensuite, nous affinerons l'analyse en faisant des régressions par région/année.

### **1) Récupération des données**
#### **a) Données météorologiques**

In [24]:
import pandas as pd

donnees_vent = pd.read_csv(r'C:\Users\peign\Desktop\ProjetPython\donnees_vent.csv', sep=",", names=['Index', 'Time', 'Windspeed'])

In [25]:
donnees_vent.head()

,Index,Time,Windspeed
0,0,"DatetimeIndex(['2011-04-23 00:00:00', '2011-04...","[2.34, 2.33, 1.8, 1.84, 1.7, 2.01, 1.35, 1.08,..."
1,1,"DatetimeIndex(['2014-04-01 00:00:00', '2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
2,2,"DatetimeIndex(['2014-04-01 00:00:00', '2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
3,3,"DatetimeIndex(['2014-04-01 00:00:00', '2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
4,4,"DatetimeIndex(['2021-11-04 00:00:00', '2021-11...","[1.61, 1.71, 1.53, 1.86, 1.82, 1.57, 1.61, 1.7..."


In [26]:
donnees_vent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7454 entries, 0 to 7453
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Index      7454 non-null   int64 
 1   Time       7454 non-null   object
 2   Windspeed  7454 non-null   object
dtypes: int64(1), object(2)
memory usage: 174.8+ KB


#### **b) Données RTE**

### **2) Régression au niveau national**
#### **Calcul des moyennes**

In [ ]:
hours = df.groupby(df.index.hour)

mean_nat = hours.mean()

#### **Régression linéaire**

In [ ]:
import statsmodels.formula.api as smf

def regression_results(data):
    model = smf.ols("electricity ~ wind", data=data).fit()
    predictions = model.predict(data)
    print(model.summary())
    
regression_results(mean_nat)  

#### **Graphiques**

In [ ]:
def plot_linear_regression(data):
    plot = sns.regplot(x="wind", y="electricity", data=data)
    sns.lmplot(x="wind", y="electricity", data=data)
    return plot

plot = plot_linear_regression(mean_nat)
plt.show()